decription is [here](https://github.com/adaptive-learning/matmat-web/blob/master/data/data_description.md)

https://www.fi.muni.cz/adaptivelearning/?a=data

In [1]:
import os
import sys
import gzip
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
from utils import *
from tqdm import tqdm

In [6]:
prefix = '/mnt/qb/work/mlcolab/hzhou52/kt/slepemapy'
log = pd.read_csv(os.path.join(prefix, 'answer.csv'), sep=';')
log.head()
# random - 0 for adaptively selected items and 1 randomly selected items

,id,user,place_asked,place_answered,type,inserted,response_time,place_map,language,options,ip_country,ip_id
0,2407,75,131,NaN,2,2013-09-26 14:45:51,8250,NaN,0,"[57, 82, 131, 77]",NaN,NaN
1,2408,75,130,69.0,1,2013-09-26 14:46:03,8526,NaN,0,"[58, 219, 69, 130]",NaN,NaN
2,2409,75,136,128.0,1,2013-09-26 14:46:10,4060,NaN,0,"[128, 136, 216, 54]",NaN,NaN
3,2410,75,126,126.0,1,2013-09-26 14:46:20,1842,NaN,0,"[153, 58, 126, 214]",NaN,NaN
4,2411,75,101,101.0,2,2013-09-26 14:46:33,10258,NaN,0,"[153, 163, 101, 195]",NaN,NaN


In [7]:
print('The columns of log are ', log.columns)
print('The length of log is {}'.format(len(log)))

print('###########################################')

print(count_unique(log, log.columns))

The columns of log are  Index(['id', 'user', 'place_asked', 'place_answered', 'type', 'inserted',
       'response_time', 'place_map', 'language', 'options', 'ip_country',
       'ip_id'],
      dtype='object')
The length of log is 10087305
###########################################
Number of unique values in id: 10087305
Number of unique values in user: 91331
Number of unique values in place_asked: 1458
Number of unique values in place_answered: 1454
Number of unique values in type: 2
Number of unique values in inserted: 7417236
Number of unique values in response_time: 123472
Number of unique values in place_map: 23
Number of unique values in language: 3
Number of unique values in options: 464053
Number of unique values in ip_country: 106
Number of unique values in ip_id: 45256
None


In [8]:
for col in log.columns:
    print(col, log[col][0:5])

id 0    2407
1    2408
2    2409
3    2410
4    2411
Name: id, dtype: int64
user 0    75
1    75
2    75
3    75
4    75
Name: user, dtype: int64
place_asked 0    131
1    130
2    136
3    126
4    101
Name: place_asked, dtype: int64
place_answered 0      NaN
1     69.0
2    128.0
3    126.0
4    101.0
Name: place_answered, dtype: float64
type 0    2
1    1
2    1
3    1
4    2
Name: type, dtype: int64
inserted 0    2013-09-26 14:45:51
1    2013-09-26 14:46:03
2    2013-09-26 14:46:10
3    2013-09-26 14:46:20
4    2013-09-26 14:46:33
Name: inserted, dtype: object
response_time 0     8250
1     8526
2     4060
3     1842
4    10258
Name: response_time, dtype: int64
place_map 0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: place_map, dtype: float64
language 0    0
1    0
2    0
3    0
4    0
Name: language, dtype: int64
options 0       [57, 82, 131, 77]
1      [58, 219, 69, 130]
2     [128, 136, 216, 54]
3     [153, 58, 126, 214]
4    [153, 163, 101, 195]
Name: options, dtype: object
ip_c

In [53]:
answer = remove_log(answer, 'student')
print(len(answer))
print(count_unique(answer, ['item', 'student']))

In [9]:
# Every item belongs to exactly one skill (mostly level 2 or level 3 skills). 
# Features 'skill_lvl_X' stands for ancestor skill of item on given level. 
# They can be used for comfortable item filtering.
item = pd.read_csv(os.path.join(prefix, 'items.csv'))#, sep=';')
item.head()

,id,question,answer,visualization,skill,skill_lvl_1,skill_lvl_2,skill_lvl_3,data
0,1,1,1,object_selection_answer,1873,1870,1871.0,1873.0,"{'answer':1,'operands':[1]}"
1,2,1,1,number_line_answer,1873,1870,1871.0,1873.0,"{'answer':1,'operands':[1]}"
2,3,1,1,object_counting,1873,1870,1871.0,1873.0,"{'answer':1,'operands':[1]}"
3,4,2,2,object_selection_answer,1874,1870,1871.0,1874.0,"{'answer':2,'operands':[2]}"
4,5,2,2,number_line_answer,1874,1870,1871.0,1874.0,"{'answer':2,'operands':[2]}"


In [25]:
print(check_nan(item))
print(count_unique(item, ['skill']))

number of NaN in column id: 0
number of NaN in column question: 0
number of NaN in column answer: 0
number of NaN in column visualization: 0
number of NaN in column skill: 0
number of NaN in column skill_lvl_1: 0
number of NaN in column skill_lvl_2: 184
number of NaN in column skill_lvl_3: 641
number of NaN in column data: 0


In [60]:
skill = pd.read_csv(os.path.join(prefix, 'skills.csv'))#, sep=';')
skill.head()
# identifier - string identifier of the skill
# name - czech name of the skill
# parent - parent skill id

,id,identifier,name,parent
0,1869,math,Vše,NaN
1,1870,numbers,Počítání,1869.0
2,1871,numbers_leq_10,Počítání do 10,1870.0
3,1872,numbers_leq_20,Počítání do 20,1870.0
4,1873,1,1,1871.0


In [61]:
print(check_nan(skill))
print(count_unique(skill, ['id']))

number of NaN in column id: 0
number of NaN in column identifier: 0
number of NaN in column name: 0
number of NaN in column parent: 1
None
number of unique value in id: 467
None


# pre-process

In [62]:
# re-index
# - skill
skill_id = list(skill['id'].unique())
skill_dict = dict(zip(skill_id, range(len(skill_id))))
skill['id'] = skill['id'].apply(lambda x: skill_dict[x])
skill['parent'][1:] = skill['parent'][1:].apply(lambda x: skill_dict[x])
skill.head()
item['skill'] = item['skill'].apply(lambda x: skill_dict[x])
item.head()

# - item
item_id = list(item['id'].unique())
item_dict = dict(zip(item_id, range(len(item_id))))
item['id'] = item['id'].apply(lambda x: item_dict[x])
answer['item'] = answer['item'].apply(lambda x: item_dict[x])

# - student
stu_id = list(answer['student'].unique())
stu_dict = dict(zip(stu_id, range(len(stu_id))))
answer['student'] = answer['student'].apply(lambda x: stu_dict[x])

,id,identifier,name,parent
0,0,math,Vše,NaN
1,1,numbers,Počítání,0.0
2,2,numbers_leq_10,Počítání do 10,1.0
3,3,numbers_leq_20,Počítání do 20,1.0
4,4,1,1,2.0


In [93]:
# create new data
df = pd.DataFrame(columns=['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time', 'correct'])
df.user_id = answer.student
df.problem_id = answer.item
item_skill = dict(zip(list(item.id), list(item.skill)))
df.skill_id = answer.item.apply(lambda x: item_skill[x])
df.timestamp = answer.time.apply(lambda t: time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')))
df.dwell_time = answer.response_time.apply(lambda t: t / 1000.0)
df.correct = answer.correct

In [94]:
df.timestamp = answer.time.apply(lambda t: time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')))

ValueError: unconverted data remains: .035462

In [95]:
try:
    df.timestamp = answer.time.apply(lambda t: time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')))
except ValueError:
    import ipdb
    ipdb.set_trace()

--Return--
None
> /tmp/ipykernel_8296/3338134284.py(5)<cell line: 1>()
      3 except ValueError:
      4     import ipdb
----> 5     ipdb.set_trace()

'2016-04-17 14:12:09'
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


In [83]:
t = answer.time[0]

In [84]:
time.strptime(t, '%Y-%m-%d %H:%M:%S')

time.struct_time(tm_year=2016, tm_mon=4, tm_mday=17, tm_hour=14, tm_min=12, tm_sec=9, tm_wday=6, tm_yday=108, tm_isdst=-1)

In [86]:
time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S'))

1460895129.0

In [88]:
from dateutil import parser

In [91]:
time.mktime(parser.parse(t))

TypeError: Tuple or struct_time argument required

In [92]:
parser.parse(t)[0]

TypeError: 'datetime.datetime' object is not subscriptable